# Compile to Custom Ring Topology

In this notebook, a .quil benchmark is compiled to a ring topology using the quil compiler and saved to an output file. The size of this ring device is of size n where n is the number of qubits used in the .quil quantum algorithm. This custom device uses the native gate library of the Rigetti QPUs.

Before running this notebook, run "qvm -S" and "quilc -S" in your terminal.

**NOTE: Depth is being used here to describe the maximum number of operations on a qubit

In [1]:
from pyquil import Program, get_qc, list_quantum_computers
from pyquil.api import QuantumComputer, ForestConnection, QVM
from pyquil.gates import *
import numpy as np
import networkx as nx
from pyquil.api import LocalQVMCompiler
from pyquil.device import NxDevice

In [2]:
def two_qubit_count(program):
    counter = 0
    if type(program) == Program:
        #rigetti circuit object
        for item in program:
            line = str(item).split(" ")
            if line[0].lower() == 'cz':
                counter = counter +1
    return counter

def compute_depth_rigetti(p):
    levels = dict.fromkeys(p.get_qubits(), 0)
    for g in p.instructions:
        if isinstance(g, Gate):
            for q in g.get_qubits():
                levels[q] += 1
    return max(levels.values())

def compile_rigetti(num_qubits,topology,program):
    if topology.lower() == 'ring':
        edge_list = []
        for i in range(0,num_qubits):
            edge_list.append((i,(i+1)%num_qubits))
        
        topology = nx.from_edgelist(edge_list)
        device = NxDevice(topology)
    
        compiler=LocalQVMCompiler("http://localhost:6000",device)
        my_qc = QuantumComputer(name='my_qc', qam=QVM(connection=ForestConnection()), 
                            device=device, compiler=compiler)
        executable = compiler.quil_to_native_quil(program) #create QC compatible specification 
        depth = compute_depth_rigetti(executable)
        volume = len(executable)-3 #subtract extra directives 
        print(executable)
        q2_count = two_qubit_count(executable)
        out_str = str(executable)
        out_str = out_str + ("#DEPTH: %s |VOL.: %s |2Q GATE COUNT: %s\n" %(depth,volume,q2_count))
        print("DEPTH: %s |VOL.: %s |2Q GATE COUNT: %s" %(depth,volume,q2_count))
        print()
        print()
        return out_str

In [3]:
# point to file that you would like to compile to a ring topology
file_in = "sample_files/barenco_tof_3.quil"
quil_in = Program(open(file_in).read())
qubits_used = max(quil_in.get_qubits())+1
topology = 'ring'

f_quil = open(file_in[0:-5]+'_cmpled.quil','w')
print("Your compiled quil program is:")
quil_out = compile_rigetti(qubits_used,topology,quil_in)


# write compiled quil to file
f_quil.write(quil_out)
f_quil.close()


Your compiled quil program is:
PRAGMA EXPECTED_REWIRING "#(0 1 2 3 4)"
RZ(-pi/4) 1
RX(pi/2) 1
CZ 1 2
RX(-pi/2) 1
CZ 0 1
RZ(1.0368263888057052) 1
RX(pi/2) 1
RZ(2.3883946005286445) 1
RX(-pi/2) 1
RZ(-2.3471249209028273) 1
RZ(2.572341968195397) 2
RX(pi/2) 2
RZ(1.9892562743228366) 2
RX(-pi/2) 2
CZ 1 2
RX(-pi/2) 1
RZ(-2.593524142005582) 2
RX(pi/2) 2
CZ 1 2
RX(pi/2) 1
RX(-pi/2) 2
CZ 1 2
RZ(-pi/2) 0
RX(-pi/2) 0
RZ(-1.5561137177769926) 1
RX(pi/2) 1
RZ(2.2632051676644607) 1
RX(-pi/2) 1
RZ(0.022996919295874076) 1
CZ 1 0
RZ(2.4517365666435613) 1
RX(-pi/2) 1
RZ(-pi/4) 1
RX(pi/2) 1
RZ(-0.9895615549140664) 2
RX(pi/2) 2
RZ(1.0609879216748244) 2
RX(-pi/2) 2
CZ 1 2
RZ(2.905070274997979) 0
RX(pi/2) 0
RZ(1.2891488963023914) 0
RX(-pi/2) 0
RZ(1.9973655005289093) 0
RZ(0.9738018203185806) 1
RX(pi/2) 1
RZ(2.006085328923567) 1
RX(-pi/2) 1
RZ(-1.867086350513131) 1
CZ 1 0
RZ(2.818965001530187) 0
RX(pi/2) 0
RX(-pi/2) 1
CZ 1 0
RX(-pi/2) 0
RX(pi/2) 1
CZ 1 0
RZ(pi/2) 4
RX(pi/2) 4
CZ 3 4
RZ(-1.010150611650983) 0
RX(pi